In [1]:
import pandas as pd

# 1 Limpiando data

In [2]:
df_historical_data = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')

## 1.1 Limpiando df_fixture

In [3]:
# Eliminamos la columna score, renombramos homes y guardamos los cambios
df_fixture.drop('score', axis=1, inplace=True)
df_fixture.rename(columns={"homes": "home"}, inplace=True)

#str hace referencia a la cadena, strip elimina los espacios en blanco
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

## 1.2 Limpiando df_missing_data y agregandolo a df_historical_data

En el video se usó para chequear la data faltante, pero en nuestro caso no fue necesario ya que la página cambió y los datos se rescataron limpios

In [4]:
# df_historical_data[df_historical_data['home'].isnull()]
# df_historical_data.dropna(inplace=True)
df_historical_data[df_historical_data['home'].isnull()]

,home,away,score,year


A continuación, en el video se concatenaron los df de la data faltante, con la data histórica (archivos resultantes de los archivos 2 y 3).

**inplace=True** sirve para guardar los cambios en el dataframe al aplicar una función

In [5]:
# pd.concat([df_data_faltante, df_historical_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True) # Elimina los duplicados
df_historical_data.sort_values('year', inplace=True) # Ordena el dataframe por año

In [6]:
# Test de regex de python
aux = df_historical_data.copy()
aux['score'] = df_historical_data['score'].str.replace('[^\d–-]', '', regex=True)
aux[aux['home'].str.contains('West Germany') &
    aux['away'].str.contains('France')]

,home,away,score,year
326,West Germany,France,3-3,1982


## 1.3 Limpiando df_historical_data

Ahora falta limpiar un partido donde hubo un "walkover", es decir, el partido no tuvo un resultado.
Esta data no ayuda al análisis por lo que es preferible eliminarla, en este caso le corresponde al partido entre Suecia y Austria

In [7]:
#str hace referencia a la cadena, strip elimina los espacios en blanco
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

In [8]:
# Las siguientes dos formas son válidas para obtener los partidos resultantes en walkover
# df_historical_data[df_historical_data['home'].str.contains('Sweden') & 
#                    df_historical_data['away'].str.contains('Austria')]

delete_index = df_historical_data[df_historical_data['score'].str.contains('w/o')].index

In [9]:
# Eliminamos y guardamos los cambios
df_historical_data.drop(delete_index, inplace=True)

In [10]:
# Comprobamos que se haya eliminado
df_historical_data[df_historical_data['score'].str.contains('w/o')]

,home,away,score,year


Ahora procedemos a limpiar el score buscando los caracteres que no son dígitos mediante la expresión "[^\d–]" y los reemplazamos con un espacio vacío

In [11]:
# [^\d–-] encuentra caracteres que no sean dígitos o contengan los símbolos "–" y "-"
df_historical_data[df_historical_data['score'].str.contains('[^\d–-]')]

,home,away,score,year
27,Italy,Spain,1–1 (a.e.t.),1934
34,Italy,Czechoslovakia,2–1 (a.e.t.),1934
24,Austria,France,3–2 (a.e.t.),1934
48,Brazil,Czechoslovakia,1–1 (a.e.t.),1938
42,Czechoslovakia,Netherlands,3–0 (a.e.t.),1938
...,...,...,...,...
866,Croatia,England,2–1 (a.e.t.),2018
855,Spain,Russia,1–1 (a.e.t.),2018
856,Croatia,Denmark,1–1 (a.e.t.),2018
860,Colombia,England,1–1 (a.e.t.),2018


In [12]:
# reemplazamos los caracteres residuales de la columna score
# regex=True hace referencia a la secuencia del tipo [^\d–-] para encontrar caracteres 
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–-]', '', regex=True)

El siguiente paso es separar el score en "HomeScore" y "AwayScore" mediante la función str.split()

In [32]:
df_historical_data[["HomeScore", "AwayScore"]] = df_historical_data['score'].str.split('–|-', expand=True)

In [33]:
df_historical_data.drop('score', axis=1, inplace=True)

In [34]:
df_historical_data.rename(columns={'home':'HomeTeam', 'away':'AwayTeam', 'year':'Year'}, inplace=True)

Finalmente vamos a asignar los tipos de datos correctos a cada columna para evitar inconsistencias en los datos, como por ejemplo tratar como "object" los datos de las columnas "HomeScore" y "AwayScore"

In [43]:
df_historical_data.dtypes
# df_historical_data[df_historical_data['AwayScore'].isnull()]

HomeTeam     object
AwayTeam     object
Year          int32
HomeScore     int32
AwayScore     int32
dtype: object

In [42]:
df_historical_data = df_historical_data.astype({'HomeScore': int, 'AwayScore': int, 'Year': int})

Creamos una nueva columna sumando los goles totales del partido. Destacar que el paso previo fue necesario debido a que no se pueden sumar datos de distintos tipos.

In [44]:
df_historical_data['TotalScore'] = df_historical_data['HomeScore'] + df_historical_data['AwayScore']

In [45]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeScore,AwayScore,TotalScore
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
831,Brazil,Costa Rica,2018,2,0,2
832,Serbia,Switzerland,2018,1,2,3
833,Serbia,Brazil,2018,0,2,2
835,Germany,Mexico,2018,0,1,1


# 2 Exportar dataframes limpios

In [46]:
df_historical_data.to_csv('clean_fifa_worldcup_historical_data.csv', index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index=False)

Algo que se suele hacer es comprobar la cantidad de elementos del dataframe, para verficar si se aún requieren ser limpiados.

En este caso vamos a mostrar la cantidad de partidos jugados por cada año.

In [55]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64
